In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install DeepTools (needed for computeMatrix and plotHeatmap)
!pip install deeptools

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.8/208.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.8/185.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 51.0 MB/s eta 0:00:00
  Created wheel for deeptoolsintervals: filename=deeptoolsintervals-0.1.9-cp311-cp311-linux_x86_64.whl size=113421 sha256=d82bcd8d52ce26495b1e22ceab0bdb11376b6a3556a8563b8fd121abae05a621
  Stored in directory: /root/.cache/pip/wheels/b5/09/8e/8589e68f9abaadf58a829c4415816154f5b1a43bd75faa13e7
Successfully built deeptoolsintervals


In [ ]:
#v3 custom labels

# Step 1: Specify the paths to your .bw files in Google Drive
bw_files = [
    '/content/drive/MyDrive/V2CHIPDATA/LNCaP_shME3_78_p97_7_AR_IP_DMSO_EtOH.chip.bw',  # Path to Control file
    '/content/drive/MyDrive/V2CHIPDATA/LNCaP_shME3_78_p97_7_AR_IP_DMSO_R1881.chip.bw',  # Path to Treatment 1
    '/content/drive/MyDrive/V2CHIPDATA/LNCaP_shME3_78_p97_7_AR_IP_DOX_EtOH.chip.bw',  # Path to Treatment 2
    '/content/drive/MyDrive/V2CHIPDATA/LNCaP_shME3_78_p97_7_AR_IP_DOX_R1881.chip.bw'   # Path to Treatment 3
]

# Step 2: Ask for custom labels for each condition
custom_labels = []
for i, bw_file in enumerate(bw_files):
    label = input(f"Enter a custom label for {bw_file.split('/')[-1]}: ")
    custom_labels.append(label)

# Step 3: Proceed with your existing code
import pandas as pd
from google.colab import files

# Your existing code to upload and process the BED file
bed_uploaded = files.upload()
bed_filename = next(iter(bed_uploaded))

# Process top 1000 peaks
bed_df = pd.read_csv(bed_filename, sep='\t', header=None)
bed_df_sorted = bed_df.sort_values(by=4, ascending=False).head(1000).copy()

if bed_df.shape[1] >= 10:
    bed_df_sorted[1] = bed_df_sorted[1] + bed_df_sorted[9] - 1
else:
    bed_df_sorted[1] = ((bed_df_sorted[1] + bed_df_sorted[2]) // 2).astype(int)

bed_df_sorted[2] = bed_df_sorted[1] + 1
top_bed_file = "top_1000_summits.bed"
bed_df_sorted[[0, 1, 2]].to_csv(top_bed_file, sep='\t', header=False, index=False)

# Run computeMatrix
output_matrix = "matrix_TFA.gz"
bw_file_args = " ".join(bw_files)

!computeMatrix reference-point \
  -S {bw_file_args} \
  -R {top_bed_file} \
  --referencePoint center \
  -a 1000 -b 1000 \
  --binSize 50 \
  --skipZeros \
  -out {output_matrix}

# Plot the heatmap with custom labels
!plotHeatmap \
  -m {output_matrix} \
  -out chipseq_heatmap.png \
  --colorMap "YlGnBu" \
  --refPointLabel "Peak Summit" \
  --samplesLabel {' '.join([f'"{label}"' for label in custom_labels])} \
  --heatmapHeight 12 \
  --heatmapWidth 6

# Download the output heatmap
files.download("chipseq_heatmap.png")

Enter a custom label for LNCaP_shME3_78_p97_7_AR_IP_DMSO_EtOH.chip.bw: AR DMSO/ETOH
Enter a custom label for LNCaP_shME3_78_p97_7_AR_IP_DMSO_R1881.chip.bw: AR DMSO/R1881
Enter a custom label for LNCaP_shME3_78_p97_7_AR_IP_DOX_EtOH.chip.bw: AR DOC/ETOH
Enter a custom label for LNCaP_shME3_78_p97_7_AR_IP_DOX_R1881.chip.bw: AR DOX/R1881


Saving LNCaP_p105_AR_IP_Whole.peaks.bed to LNCaP_p105_AR_IP_Whole.peaks.bed

The following chromosome names did not match between the bigwig files
chromosome	length
     KI270548.1	      1599
     KI270752.1	     27745


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>